# Numerical integration of double variable polynomials
This example uses the `integrate_2d` function to calculate
the integral
$$\int_{-1}^{1} P(x, y) \partial x \partial y$$
where $P_0(x)$ and $P_1(y)$ are polynomials.

For this example,
$$P(x, y) = L^2_i(\xi) L^2_i(\eta)$$

Where,

$L_i$, $i \in \{0 ... 7\}$, are the Lagrange polynomials created using $8$ Legendre-Gauss-Lobatto(LGL) points.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

import numpy as np
import arrayfire as af

from dg_maxwell import params
from dg_maxwell import utils
from dg_maxwell import lagrange
from dg_maxwell import wave_equation_2d as w2d

af.set_backend(params.backend)

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


In [2]:
# Define the polynomials
degree_xi = 6
p_xi = af.flip(af.range(degree_xi) + 1)
p_xi = af.moddims(p_xi, d0 = 2, d1 = 3)

degree_eta = 6
p_eta = af.flip(af.range(degree_eta) + 1)
p_eta = af.moddims(p_eta, d0 = 2, d1 = 3)


In [3]:
print(p_xi)

arrayfire.Array()
Type: float

[2 3 1 1]
    6.0000     4.0000     2.0000 
    5.0000     3.0000     1.0000 




In [4]:
p_xi_eta = utils.polynomial_product_coeffs(af.reorder(p_xi, 1, 2, 0), af.reorder(p_eta, 1, 2, 0))
poly_xi_eta = p_xi_eta[:, :, 1]

In [96]:
utils.integrate_2d_multivar_poly(p_xi_eta[:, :, 1], N_quad = 9, scheme = 'gauss')

28.44444444444407

In [97]:
# Gauss-Legendre quadrature
N_quad = 9

shape_poly_2d = utils.shape(poly_xi_eta)
xi_gauss  = af.np_to_af_array(lagrange.gauss_nodes(N_quad))
eta_gauss = af.np_to_af_array(lagrange.gauss_nodes(N_quad))

Xi, Eta = utils.af_meshgrid(xi_gauss, eta_gauss)

Xi  = af.flat(Xi)
Eta = af.flat(Eta)

w_i = lagrange.gaussian_weights(N_quad)
w_j = lagrange.gaussian_weights(N_quad)

W_i, W_j = utils.af_meshgrid(w_i, w_j)

W_i = af.tile(af.flat(W_i), d0 = 1, d1 = shape_poly_2d[2])
W_j = af.tile(af.flat(W_j), d0 = 1, d1 = shape_poly_2d[2])

P_xi_eta_quad_val = af.transpose(utils.polyval_2d(poly_xi_eta, Xi, Eta))

integral = af.sum(W_i * W_j * P_xi_eta_quad_val, dim = 0)
print(integral)

arrayfire.Array()
Type: double

[1 1 1 1]
   28.4444 



In [99]:
shape_poly_2d = utils.shape(poly_xi_eta)

xi_LGL  = lagrange.LGL_points(N_quad)
eta_LGL = lagrange.LGL_points(N_quad)

Xi, Eta = utils.af_meshgrid(xi_LGL, eta_LGL)

Xi  = af.flat(Xi)
Eta = af.flat(Eta)

w_i = lagrange.lobatto_weights(N_quad)
w_j = lagrange.lobatto_weights(N_quad)

W_i, W_j = utils.af_meshgrid(w_i, w_j)

W_i = af.tile(af.flat(W_i), d0 = 1, d1 = shape_poly_2d[2])
W_j = af.tile(af.flat(W_j), d0 = 1, d1 = shape_poly_2d[2])

P_xi_eta_quad_val = af.transpose(utils.polyval_2d(poly_xi_eta, Xi, Eta))

integral = af.sum(W_i * W_j * P_xi_eta_quad_val, dim = 0)

print(integral)


arrayfire.Array()
Type: double

[1 1 1 1]
   28.4444 



/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


In [5]:
#Using the integrate_2d_multival_poly function
print(utils.integrate_2d_multivar_poly(p_xi_eta, N_quad = 9, scheme = 'gauss'))
print(utils.integrate_2d_multivar_poly(p_xi_eta, N_quad = 9, scheme = 'lobatto'))

arrayfire.Array()
Type: double

[2 1 1 1]
   64.0000 
   28.4444 


arrayfire.Array()
Type: double

[2 1 1 1]
   64.0000 
   28.4444 




/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
